In [1]:
import torch
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import plotly.express as px
import pandas as pd
import logging
import os
from sklearn.metrics import f1_score
import torch.nn.functional as F

from conv_model import Model
from dataset_preprocessing import Paths, Dataset, Dataloader, Dataloader_parts
from metrics import confusion_matrix, accuracy, accuracy_per_class

/home/noonmare/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/noonmare/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class Training:
    """Parameters of training process"""

    training_mode = False

    #dataset parameters
    pandora_18k = Dataset(path_to_the_dataset=Paths.pandora_18k)
    num_classes = pandora_18k.number_of_classes
    batch_size = 24

    train_lu = Paths.pandora_18k + "Train_lu/"
    train_ld = Paths.pandora_18k + "Train_ld/"
    train_ru = Paths.pandora_18k + "Train_ru/"
    train_rd = Paths.pandora_18k + "Train_rd/"
    train_c = Paths.pandora_18k + "Train_c/"

    valid_lu = Paths.pandora_18k + "Validation_lu/"
    valid_ld = Paths.pandora_18k + "Validation_ld/"
    valid_ru = Paths.pandora_18k + "Validation_ru/"
    valid_rd = Paths.pandora_18k + "Validation_rd/"
    valid_c = Paths.pandora_18k + "Validation_c/"

    test_lu = Paths.pandora_18k + "Test_lu/"
    test_ld = Paths.pandora_18k + "Test_ld/"
    test_ru = Paths.pandora_18k + "Test_ru/"
    test_rd = Paths.pandora_18k + "Test_rd/"
    test_c = Paths.pandora_18k + "Test_c/"


    #device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    #logging
    logging_file = "logs/inception_v3.log"

    #model paths
    model_path = Paths.pandora_18k + 'Conv_models/Inception-V3/'
    model_save_path = model_path + 'inc_v3_model.pth' 

    #model initialization parameters
    model_init_kwargs = {

    'device' : device,
    'num_classes' : num_classes,

    #convolutional network basic parameters
    'conv_name' : 'Inception-V3',
    'conv_model' : models.inception_v3(weights='IMAGENET1K_V1'),
    'input_shape' :  (299, 299)

    }
    
    #model training parameters
    model_training_kwargs = {

    'device' : device,
    'num_epochs' : 45,
    'criterion' : (nn.CrossEntropyLoss(), 'Cross-Entropy'), 

    #optimizer parameters
    'optimizer' : (optim.SGD, 'SGD'),
    'learn_rate' :  0.0002,
    'momentum' :  0.888,
    'nesterov' : True,
    #scheduler parameters
    'scheduler' : lr_scheduler.StepLR,
    'step_size' :  10,
    'gamma' :  0.9
    }

    #probability vectors paths
    prb_vec_train = model_path + 'train.csv'
    prb_vec_valid = model_path + 'valid.csv'
    prb_vec_test = model_path + 'test.csv'

In [3]:
torch.manual_seed(42)
cudnn.benchmark = False
logging.basicConfig(level=logging.INFO, filename=Training.logging_file,filemode="a",
                    format="%(asctime)s %(levelname)s %(message)s")

model = Model(Training.model_init_kwargs)

dataloader = Dataloader(model, Training.pandora_18k.train_path, Training.pandora_18k.val_path, Training.pandora_18k.test_path, Training.batch_size)

if Training.training_mode:

    Training.model_training_kwargs['dataloader'] = dataloader

    model = model.fit(Training.model_training_kwargs)

    torch.save(model.conv_nn.state_dict(), Training.model_save_path)

else:
    conv_nn = Training.model_init_kwargs.get('conv_model')                                                                                                                                                                                                                                                                                                                                         
    conv_nn.load_state_dict(torch.load(Training.model_save_path))
    conv_nn.eval()

    model.conv_nn = conv_nn

Inception-V3 initialized


#### Total accuracy и accuracy per each class

In [4]:
pred, target = model.predict(device=Training.device, dataiter=iter(dataloader.dataloaders['test']), classes=Training.pandora_18k.classes)

print(f'Total accuracy: {accuracy(pred, target):.1f} %')

acc_per_class = accuracy_per_class(pred, target, Training.pandora_18k.classes)

for style, acc in acc_per_class.items():
    print(f'Accuracy for {style}: {acc:.1f} %')

Total accuracy: 64.4 %
Accuracy for 01_Byzantin_Iconography: 95.3 %
Accuracy for 02_Early_Renaissance: 79.7 %
Accuracy for 03_Northern_Renaissance: 65.0 %
Accuracy for 04_High_Renaissance: 59.9 %
Accuracy for 05_Baroque: 60.0 %
Accuracy for 06_Rococo: 46.1 %
Accuracy for 07_Romanticism: 41.1 %
Accuracy for 08_Realism: 61.7 %
Accuracy for 09_Impressionism: 71.5 %
Accuracy for 10_Post_Impressionism: 38.3 %
Accuracy for 11_Expressionism: 39.7 %
Accuracy for 12_Symbolism: 57.3 %
Accuracy for 13_Fauvism: 50.6 %
Accuracy for 14_Cubism: 77.4 %
Accuracy for 15_Surrealism: 58.7 %
Accuracy for 16_AbstractArt: 70.2 %
Accuracy for 17_NaiveArt: 67.6 %
Accuracy for 18_PopArt: 67.8 %
Accuracy for 19_ChineseArt: 67.8 %
Accuracy for 20_JapaneseArt: 97.8 %


#### Mean min-3 accuracy 

In [5]:
x = sorted(list(acc_per_class.items()), key=lambda x : x[1])[:3]

print(f"Mean accuracy for min 3 styles : {sum([el[1] for el in x]) / 3:.1f} %")

for style, acc in x:
    print(f'Accuracy for {style}: {acc:.1f} %')

Mean accuracy for min 3 styles : 39.7 %
Accuracy for 10_Post_Impressionism: 38.3 %
Accuracy for 11_Expressionism: 39.7 %
Accuracy for 07_Romanticism: 41.1 %


: 